<a href="https://colab.research.google.com/github/pfelesova/dat.veda/blob/main/2021_11_28_How_To_Train_CNN_Classifier_With_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How To Train CNN Classifier With TF

This notebook demonstrates how to use `genomic_benchmarks` to train a neural network classifier on one of its benchmark datasets [human_nontata_promoters](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/tree/main/docs/human_nontata_promoters).

In [1]:
#If you work in Google Colaboratory - uncomment the following line to install the package to your virtual machine  
!pip install -qq tensorflow_addons genomic-benchmarks

     |████████████████████████████████| 1.1 MB 5.3 MB/s 
     |████████████████████████████████| 2.3 MB 34.0 MB/s 
     |████████████████████████████████| 596 kB 48.8 MB/s 
     |████████████████████████████████| 271 kB 44.5 MB/s 
     |████████████████████████████████| 160 kB 45.0 MB/s 


# Data download

With the function `download_dataset` downloads, we can download full-sequence form of the benchmark, splitted into train and test sets, one folder for each class.

In [2]:
from pathlib import Path
import tensorflow as tf
import numpy as np

from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer, binary_f1_score
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded('demo_human_or_worm'):
    download_dataset('demo_human_or_worm')

/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Done.
Unzipping...Done.


In [3]:
info('demo_human_or_worm', 0)

Dataset `demo_human_or_worm` has 2 classes: human, worm.

All lenghts of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.


,train,test
human,37500,12500
worm,37500,12500


## TF Dataset object

To train the model with TensorFlow, we must create a TF Dataset. Because the directory structure of our benchmarks is ready for training, we can just call `tf.keras.preprocessing.text_dataset_from_directory` function as follows.

In [4]:
BATCH_SIZE = 128
SEQ_PATH = Path.home() / '.genomic_benchmarks' / 'demo_human_or_worm'
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 75000 files belonging to 2 classes.


## Text vectorization

To convert the strings to tensors, we internally use TF `TextVectorization` layer and splitting to characters.

In [5]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
vectorize_layer.get_vocabulary()

['', '[UNK]', 'a', 't', 'c', 'g', 'n']

In [6]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

To get a baseline (other models can be compared to) we ship a package with [a simple CNN model](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/blob/main/src/genomic_benchmarks/models/tf.py). We have vectorized the dataset before training the model to speed up the process.

In [7]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.2), binary_f1_score])

In [11]:
EPOCHS = 10

history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
586/586 [==============================] - 26s 44ms/step - loss: 0.1260 - binary_accuracy: 0.9506 - f1_score: 0.9482
Epoch 2/10
586/586 [==============================] - 27s 45ms/step - loss: 0.1230 - binary_accuracy: 0.9513 - f1_score: 0.9493
Epoch 3/10
586/586 [==============================] - 27s 46ms/step - loss: 0.1215 - binary_accuracy: 0.9519 - f1_score: 0.9501
Epoch 4/10
586/586 [==============================] - 26s 45ms/step - loss: 0.1201 - binary_accuracy: 0.9525 - f1_score: 0.9505
Epoch 5/10
586/586 [==============================] - 26s 44ms/step - loss: 0.1184 - binary_accuracy: 0.9528 - f1_score: 0.9507
Epoch 6/10
586/586 [==============================] - 26s 44ms/step - loss: 0.1171 - binary_accuracy: 0.9540 - f1_score: 0.9517
Epoch 7/10
586/586 [==============================] - 26s 44ms/step - loss: 0.1158 - binary_accuracy: 0.9538 - f1_score: 0.9518
Epoch 8/10
586/586 [==============================] - 26s 44ms/step - loss: 0.1156 - binary_accuracy: 0.

## Evaluation on the test set

Finally, we can do the same pre-processing for the test set and evaluate the F1 score of our model.

In [12]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES,
    shuffle=False)

test_ds = test_dset.map(vectorize_text)

Found 25000 files belonging to 2 classes.


In [13]:
model.evaluate(test_ds)

196/196 [==============================] - 8s 39ms/step - loss: 0.1502 - binary_accuracy: 0.9432 - f1_score: 0.9406


[0.1501772552728653, 0.9432399868965149, 0.9406252503395081]

In [ ]:
# 0.16, 0.94, 0.94
# 0.25, 0.91, 0.92
# 0.15, 0.94, 0.94